In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
conn=wrds.Connection()

Enter your WRDS username [alan]:kouzhizhuo
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [2]:
### CRSP part
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc, a.hsiccd
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1990' and '12/31/2019'
                      and b.exchcd between 1 and 3
             
                      """) 


# change variable format
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)
# SIC code 
crsp_m=crsp_m[(crsp_m['hsiccd']<6000)|(crsp_m['hsiccd']>6999)]


# Line up date to end of month
crsp_m['date']=pd.to_datetime(crsp_m['date'])
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """)
dlret.permno=dlret.permno.astype(int)
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1
crsp['me']=crsp['prc'].abs()*crsp['shrout'] # calculate market equity
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

### Aggregate About Market Cap 
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()
# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)
# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

# keep December as market cap
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month


### July to June dates
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()

# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)

# found out lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

# baseline me
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

# overall result
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

# Information of June
crsp_jun = crsp3[crsp3['month']==6]
crsp_jun=crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','retadj','me','wt','cumretx','mebase','lme']]
crsp_jun=crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()



In [3]:
###Annual Block of compustat 
comp = conn.raw_sql("""
                    select gvkey, datadate, at, pstkl, txditc,
                    pstkrv, seq, pstk
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/1990'
                    """)




comp['datadate']=pd.to_datetime(comp['datadate']) #convert datadate to date fmt
comp['year']=comp['datadate'].dt.year

#years in Compustat
comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()

# lag the total assest
comp=comp.sort_values(by=['gvkey','datadate'])
comp['lat']=comp.groupby(['gvkey'])['at'].shift(1)

#I/A function
comp['I/A']=(comp['at']/comp['lat'])-1
comp['year']=comp['year']+1

In [4]:
### Count Number of Years in Compustat
comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()
comp=comp[['gvkey','datadate','year','I/A','count']]

In [5]:
###Quarter Block
cp = conn.raw_sql("""
                    select gvkey, rdq, datadate, ibq, atq, seqq, txditcq, ceqq, pstkq, 
                    ltq, pstkrq, exchg
                    from comp.fundq
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/1990'
                    """)

cp['rdq']=pd.to_datetime(cp['rdq']) #convert datadate to date fmt
cp['datadate']=pd.to_datetime(cp['datadate']) #convert datadate to date fmt

In [6]:
#shareholders' equity
cp['se']=np.where(cp['seqq'].isnull(), (cp['ceqq']+cp['pstkq']), cp['seqq'])
cp['se']=np.where(cp['se'].isnull(),(cp['atq']-cp['ltq']), cp['se'])

In [7]:
### Choose Preferrerd Stock
cp['ps']=np.where(cp['pstkrq'].isnull(), cp['pstkq'], cp['pstkrq'])
cp['ps']=np.where(cp['ps'].isnull(),0,cp['ps'])

cp['txditcq']=cp['txditcq'].fillna(0)

# create book equity
cp['be']=cp['se']+cp['txditcq']-cp['ps']
cp['be']=np.where(cp['be']>0, cp['be'], np.nan)


# lag book equity
cp=cp.sort_values(by=['gvkey','rdq'])
cp['lbe']=cp.groupby(['gvkey'])['be'].shift(1)

In [8]:
#create ROE
cp['ROE']=cp['ibq']/cp['lbe']

#report date of the month
cp['rdqend']=cp['rdq']+MonthEnd(0)

#lag report date
cp=cp.sort_values(by=['gvkey','datadate'])
cp['lrdq']=cp.groupby(['gvkey'])['rdq'].shift(1)
cp['llrdq']=cp.groupby(['gvkey'])['lrdq'].shift(1)

In [9]:
import datetime
from datetime import timedelta
cp=cp.dropna(subset=['lrdq'])
cp['datediff']=cp['datadate']-cp['lrdq']
cp['datediff']=cp['datediff'].dt.days
cp['datediff']=np.where(cp['datediff']<0, (cp['datadate']-cp['llrdq']).dt.days, cp['datediff'])
cp=cp[(cp['datediff']<=183)&(cp['datediff']>=0)]

cp=cp[['gvkey','datadate','ROE','rdq','rdqend','exchg','lrdq','be']]

In [10]:
###CCM Part
import pyreadstat
ccm, meta = pyreadstat.read_sas7bdat('ccmlinktable.sas7bdat')
ccm.columns= ccm.columns.str.lower()
ccm=ccm[['gvkey','lpermno', 'linktype', 'linkprim', 'linkdt', 'linkenddt']]
ccm=ccm.rename(columns={'lpermno':'permno'})
ccm=ccm[(ccm['linkprim'] =='C')|(ccm['linkprim'] =='P')]
ccm=ccm[(ccm['linktype'] !='NR') & (ccm['linktype']  !='NU')]

ccm['linkdt'] = pd.to_timedelta(ccm['linkdt'], unit='D') + pd.Timestamp('1960-01-01')
ccm['linkenddt'] = pd.to_timedelta(ccm['linkenddt'], unit='D') + pd.Timestamp('1960-01-01')

ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [11]:
### merge I/A
ccm1=pd.merge(comp[['gvkey','datadate','I/A', 'count']],ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)
    # set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
ccm2=ccm2[['gvkey','permno','datadate','yearend', 'jdate','I/A', 'count']]

# merge size
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])

In [12]:
# select NYSE as breakpoint
nyse=ccm_jun[(ccm_jun['exchcd']==1) & (ccm_jun['me']>0) & (ccm_jun['count']>1) & ((ccm_jun['shrcd']==10) | (ccm_jun['shrcd']==11))]
# size breakpoint
nyse_sz=nyse.groupby(['jdate'])['me'].median().to_frame().reset_index().rename(columns={'me':'sizemedn'})
# IA breakpoint
nyse_ia=nyse.groupby(['jdate'])['I/A'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_ia=nyse_ia[['jdate','30%','70%']].rename(columns={'30%':'IA30', '70%':'IA70'})
nyse_breaks = pd.merge(nyse_sz, nyse_ia, how='inner', on=['jdate'])
# join back size and IA breakdown
ccm1_jun = pd.merge(ccm_jun, nyse_breaks, how='left', on=['jdate'])

#assign size to B&M
def sz_bucket(row):
    if row['me']==np.nan:
        value=''
    elif row['me']<=row['sizemedn']:
        value='S'
    else:
        value='B'
    return value

#assign I/A to H&M&L
def ia_bucket(row):
    if row['I/A']<=row['IA30']:
        value = 'L'
    elif row['I/A']<=row['IA70']:
        value='M'
    elif row['I/A']>row['IA70']:
        value='H'
    else:
        value=''
    return value

In [13]:
### merge ccm & crsp
cr3=crsp3[['permno','date','jdate']]
ccmcrsp=pd.merge(cr3,ccm,how='left',on=['permno'])
# set link date bounds
ccmcrsp2=ccmcrsp[(ccmcrsp['jdate']>=ccmcrsp['linkdt'])&(ccmcrsp['jdate']<=ccmcrsp['linkenddt'])]
ccmcrsp2=ccmcrsp2[['gvkey','permno','date', 'jdate']]

#merge compustata quaterly
ccmcp=pd.merge(ccmcrsp2, cp, how='left', on=['gvkey'])
ccmcp2=ccmcp[(ccmcp['jdate']>=ccmcp['lrdq'])&(ccmcp['jdate']<ccmcp['rdq'])]
ccmcp2=ccmcp2.rename(columns={'lrdq':'matchrdq'})
ccmcp3=ccmcp2[['permno','gvkey','matchrdq','date','jdate']]

cp2=cp.copy()
cp2['matchrdq']=cp2['rdq']
ccmcp4=pd.merge(ccmcp3, cp2, how='left', on=['gvkey','matchrdq'])

In [14]:
###NYSE as breakpoint
nyse2=ccmcp4[ccmcp4['exchg']==11]

# ROE breakpoint
nyse_roe=nyse2.groupby(['jdate'])['ROE'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_roe=nyse_roe[['jdate','30%','70%']].rename(columns={'30%':'roe30', '70%':'roe70'})
# join back ROE
ccmcp5 = pd.merge(ccmcp4, nyse_roe, how='left', on=['jdate'])

#assign ROE to X&Z&D
def roe_bucket(row):
    if row['ROE']<=row['roe30']:
        value = 'X'
    elif row['ROE']<=row['roe70']:
        value='Z'
    elif row['ROE']>row['roe70']:
        value='D'
    else:
        value=''
    return value

In [15]:
###factors
# assign size portfolio
ccm1_jun['szport']=np.where((ccm1_jun['me']>0)&(ccm1_jun['count']>=1), ccm1_jun.apply(sz_bucket, axis=1), '')
# assign book-to-market portfolio
ccm1_jun['iaport']=np.where((ccm1_jun['me']>0)&(ccm1_jun['count']>=1), ccm1_jun.apply(ia_bucket, axis=1), '')
# create positiveme and nonmissport variable
ccm1_jun['posme']=np.where((ccm1_jun['me']>0)&(ccm1_jun['count']>=1), 1, 0)
ccm1_jun['nonmissport']=np.where((ccm1_jun['iaport']!=''), 1, 0)

# store portfolio assignment as of June
june=ccm1_jun[['permno','date', 'jdate', 'iaport','szport','posme','nonmissport']]
june['ffyear']=june['jdate'].dt.year

#assign roe portfolio
ccmcp5['roeport']=np.where(ccmcp5['ROE'].isnull(),'', ccmcp5.apply(roe_bucket, axis=1))
ccmcp5['nonmiss']=np.where((ccmcp5['roeport']!=''), 1, 0)
ccmcp5=ccmcp5[['permno','date','jdate','roeport','nonmiss']]
ccmcp5['usedate']=ccmcp5['jdate']+MonthEnd(1)
ccmcp5=ccmcp5.rename(columns={'jdate':'fmdate'})


###merge monthly
crsp3 = crsp3[['date','permno','shrcd','exchcd','retadj','me','wt','cumretx','ffyear','jdate']]
ccm_3=pd.merge(crsp3, 
        june[['permno','ffyear','szport','iaport','posme','nonmissport']], how='left', on=['permno','ffyear'])
ccm_4=pd.merge(ccm_3, ccmcp5, how='left', left_on=['permno','jdate'], right_on=['permno','usedate'])

#keep criteria data
ccm_5=ccm_4[(ccm_4['wt']>0)& (ccm_4['posme']==1) & (ccm_4['nonmissport']==1) & (ccm_4['nonmiss']==1)&
          ((ccm_4['shrcd']==10) | (ccm_4['shrcd']==11))]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [17]:
###Form Q Factors 2*3*3
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan
    
# value-weigthed return
vwret=ccm_5.groupby(['jdate','szport','iaport','roeport']).apply(wavg, 'retadj','wt').to_frame().reset_index().rename(columns={0: 'vwret'})
vwret['sbport']=vwret['szport']+vwret['iaport']+vwret['roeport']
# tranpose
Q_factors=vwret.pivot(index='jdate', columns='sbport', values='vwret').reset_index()

###create factors
Q_factors['WB']=(Q_factors['BHD']+Q_factors['BHZ']+Q_factors['BHX']+Q_factors['BMD']+Q_factors['BMZ']+Q_factors['BMX']+Q_factors['BLD']+Q_factors['BLZ']+Q_factors['BLX'])/9
Q_factors['WS']=(Q_factors['SHD']+Q_factors['SHZ']+Q_factors['SHX']+Q_factors['SMD']+Q_factors['SMZ']+Q_factors['SMX']+Q_factors['SLD']+Q_factors['SLZ']+Q_factors['SLX'])/9
Q_factors['rME'] = Q_factors['WS']-Q_factors['WB']

Q_factors['WH']=(Q_factors['BHD']+Q_factors['BHZ']+Q_factors['BHX']+Q_factors['SHD']+Q_factors['SHZ']+Q_factors['SHX'])/6
Q_factors['WL']=(Q_factors['BLD']+Q_factors['BLZ']+Q_factors['BLX']+Q_factors['SLD']+Q_factors['SLZ']+Q_factors['SLX'])/6
Q_factors['rINN'] = Q_factors['WH']-Q_factors['WL']

Q_factors['WD']=(Q_factors['BHD']+Q_factors['BMD']+Q_factors['BLD']+Q_factors['SHD']+Q_factors['SMD']+Q_factors['SLD'])/6
Q_factors['WX']=(Q_factors['BHX']+Q_factors['BMX']+Q_factors['BLX']+Q_factors['SHX']+Q_factors['SMX']+Q_factors['SLX'])/6
Q_factors['rROE'] = Q_factors['WD']-Q_factors['WX']

Q_factors=Q_factors.rename(columns={'jdate':'date'})